In [ ]:
#default_exp core
from nbdev import *

# core

> This is core functionality for RemoteConnect

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
# IN_COLAB = 'google.colab' in str(get_ipython())
# if IN_COLAB:
#     from google.colab import drive

IN_COLAB = False
try:
    from google.colab import drive

    IN_COLAB = True
except ImportError:
    IN_COLAB = False
import os
import subprocess
from pyngrok import ngrok

In [ ]:
#exports
EXTENSIONS = ["ms-python.python", "jithurjacob.nbpreviewer"]

In [ ]:
#export
def say_hello(to):
    print(f"Say hello to {to}")

In [ ]:
#hide
active_tunnels = ngrok.get_tunnels()
for tunnel in active_tunnels:
    public_url = tunnel.public_url
    print(f"Disconnecting {public_url}")
    ngrok.disconnect(public_url)
    
url = ngrok.connect(port=10000, options={"bind_tls":True})
print(f"Code server can be assesed on : {url}")

t=2020-12-03T17:01:32+0000 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040

t=2020-12-03T17:01:33+0000 lvl=eror msg="failed to auth" obj=tunnels.session err="Your account 'Rahul Saraf' is limited to 1 simultaneous ngrok client session.\nActive ngrok client sessions in region 'us':\n  - ts_1l9gfOOMIfBboacPudEkoFeii0S (34.198.112.183)\r\n\r\nERR_NGROK_108\r\n"



PyngrokNgrokError: The ngrok process errored on start: Your account 'Rahul Saraf' is limited to 1 simultaneous ngrok client session.\nActive ngrok client sessions in region 'us':\n  - ts_1l9gfOOMIfBboacPudEkoFeii0S (34.198.112.183)\r\n\r\nERR_NGROK_108\r\n.

In [ ]:
def connect_to_localhost(port):
    pass

In [ ]:
# !ssh -R 80:localhost:8080 ssh.localhost.run -y

In [ ]:
#export
def connect_to_ngrok(port):
    active_tunnels = ngrok.get_tunnels()
    for tunnel in active_tunnels:
        public_url = tunnel.public_url
        print(f"Disconnecting {public_url}")
        ngrok.disconnect(public_url)
    url = ngrok.connect(port=port, options={"bind_tls":True})
    print(f"Remote server can be assesed on : {url}")
    return url 

In [ ]:
#export
def execute_cmd(cmd):
    print(f"Executing >>> {cmd}")
    with subprocess.Popen(
        [cmd],
        shell=True,
        stdout=subprocess.PIPE,
        bufsize=1,
        universal_newlines=True,
    ) as proc:
        for line in proc.stdout:
            print(line, end="")

In [ ]:
#export
def mount_drive():
    drive.mount("/content/drive")

In [ ]:
#export
class RemoteCode:
    """ Install and launch an instance of Remote Code"""
    def __init__(self, port=10000, password=None):
        self.port = port
        self.password = password
        self._mount = IN_COLAB
        self.url = None
        self._install_code()
        self._install_extensions()
        self._start_server()
        self._run_code()
        
    def _install_code(self):
        subprocess.run(
            ["wget", "https://code-server.dev/install.sh"], stdout=subprocess.PIPE
        )
        subprocess.run(["sh", "install.sh"], stdout=subprocess.PIPE)

    def _install_extensions(self):
        for ext in EXTENSIONS:
            subprocess.run(["code-server", "--install-extension", f"{ext}"]) 
            
    def _start_server(self):
        self.url = connect_to_ngrok(self.port)
        
    def _run_code(self):
        os.system(f"fuser -n tcp -k {self.port}")
#         if IN_COLAB:
#             drive.mount("/content/drive")
        if self.password:
            code_cmd = f"PASSWORD={self.password} code-server --port {self.port} --disable-telemetry"
        else:
            code_cmd = f"code-server --port {self.port} --auth none --disable-telemetry"
        execute_cmd(code_cmd)
        
    def __str__(self):
        return f"Remote Code runnning on {self.url}:{self.port} with password:{self.password} in colab envrionment:{IN_COLAB}"
        

In [ ]:
#exports
NOTEBOOK_EXTENSIONS = ["toc2/main", "collapsible_headings/main", "execute_time/ExecuteTime", "codefolding/main"]

In [ ]:
#export
class RemoteJupyter:
    """ Install and launch an instance of Remote Jupyter"""
    def __init__(self, port=9000, ui='notebook'):
        self.port = port
        self._mount = IN_COLAB
        self.url = None
        self.ui = ui
        if IN_COLAB:
            self._install_extensions()
        self._start_server()
        self._run_jupyter()
            
    def _install_extensions(self):
        subprocess.run(["jupyter", "contrib", "nbextension","install",  "--system"], stdout=subprocess.PIPE)
        #nbextensions_configurator enable --user
        subprocess.run(["jupyter", "nbextensions_configurator","enable",  "--system"], stdout=subprocess.PIPE)
        for ext in NOTEBOOK_EXTENSIONS:
            subprocess.run(["jupyter", "nbextension", "enable", ext], stdout=subprocess.PIPE)  

    def _start_server(self):
        self.url = connect_to_ngrok(self.port)
        
    def _run_jupyter(self):
        os.system(f"fuser -n tcp -k {self.port}")
        print(self.url)
#         if IN_COLAB:
#             drive.mount("/content/drive")
        jupyter_cmd = f"jupyter {self.ui} --NotebookApp.allow_remote_access=True  --NotebookApp.disable_check_xsrf=True --ip=0.0.0.0 --port={self.port}"
        if self.ui == 'lab':
            jupyter_cmd = f"jupyter {self.ui} --ip=0.0.0.0 --port={self.port} --no-browser"
        execute_cmd(jupyter_cmd)
        
    def __str__(self):
        return f"Remote Jupyter runnning on {self.url}:{self.port} with password:{self.password} in colab envrionment:{IN_COLAB}"

In [ ]:
#export
class RemotePluto:
    """ Install and launch an instance of Remote Pluto.jl for Julia⌈"""
    def __init__(self, port=9000):
        self.port = port
        self._mount = IN_COLAB
#         if IN_COLAB:
#             self._install_extensions()
        self._start_server()
        self._run_pluto()
            
#     def _install_extensions(self):
#         subprocess.run(["jupyter", "contrib", "nbextension","install",  "--system"], stdout=subprocess.PIPE)
#         #nbextensions_configurator enable --user
#         subprocess.run(["jupyter", "nbextensions_configurator","enable",  "--system"], stdout=subprocess.PIPE)
#         for ext in NOTEBOOK_EXTENSIONS:
#             subprocess.run(["jupyter", "nbextension", "enable", ext], stdout=subprocess.PIPE)  

    def _start_server(self):
        self.url = connect_to_ngrok(self.port)
        
    def _run_pluto(self):
        os.system(f"fuser -n tcp -k {self.port}")
        print(self.url)
#         if IN_COLAB:
#             drive.mount("/content/drive")
        pluto_cmd = f'julia -e "using Pluto;Pluto.run(port={self.port})"'
#         if self.ui == 'lab':
#             jupyter_cmd = f"jupyter {self.ui} --ip=0.0.0.0 --port={self.port} --no-browser"
        execute_cmd(pluto_cmd)
        
    def __str__(self):
        return f"Remote Pluto runnning on {self.url}:{self.port} with password:{self.password} in colab envrionment:{IN_COLAB}"